In [ ]:
## script to work out SNR/CNR one by one
# load in excel sheet with voxel locations and file destinations
# work out SNR of CC, work out SNR of CAUD
# save in table 
# Compute CNR of each image by SNR(CC) - SNR(CAUD)

import nipype                    # Nipype is a pipelining tool
import nibabel as nib            # Nibabel is a library to read in nifti-files
import numpy as np               # Numpy is 'matlab for Python'
import os                        # Useful for directory specification later on
import pandas as pd              # for reading excel sheet 

pd.set_option('display.max_rows', 1000)

In [ ]:
# define Spherical function
import numpy as np

# Function to take a 3 x 3 x 3 square around the inputted point
def spherical(x, y, z, voxel_data, mean_data):
    # Iterate through all offsets in a 3x3x3 cube centered at (x, y, z)
    offsets = [-1, 0, 1]
    for dx in offsets:
        for dy in offsets:
            for dz in offsets:
                voxel_data.append(mean_data[x + dx, y + dy, z + dz])
    
    # Calculate the mean and standard deviation
    mean_vox = np.mean(voxel_data)
    stand = np.std(voxel_data)
    
    return voxel_data, mean_vox, stand

In [ ]:
data_dir = os.getcwd() # get directoy
data_dir = '/home/scotti/Documents/DatabaseImages/meta_analysis_data'

# load excel file with voxel locations and data info 
data_sheet = pd.read_excel(os.path.join(data_dir, 'vox_location_20.xlsx'), sheet_name='Sheet1')

#data_sheet = pd.read_excel('/home/scotti/Documents/Scripts/pyth_vox.xlsx', sheetname='Sheet1') # load excel file

file_name = data_sheet.File            # name of files as variable

CCx = data_sheet.CC_x                       # x coordinate corpus callosum
CCy = data_sheet.CC_y                       # y coordinate corpus callosum
CCz = data_sheet.CC_z                       # z coordinate corpus callosum
CAUDLx = data_sheet.CAUD_L_x                  # x coordinate left caudate nucleus
CAUDLy = data_sheet.CAUD_L_y                  # y coordinate left caudate nucleus
CAUDLz = data_sheet.CAUD_L_z                  # z coordinate left caudate nucleus
CAUDRx = data_sheet.CAUD_R_x                # x coordinate right CN
CAUDRy = data_sheet.CAUD_R_y                # y coordinate right CN
CAUDRz = data_sheet.CAUD_R_z                # z coordniate right CN

SNR_data_CC = []                            # create frame for SNR-cc
SNR_CC_sd = []                              # create frame for sd of CC
voxmean_CC = []

SNR_data_CAUDL = []                         # create frame for SNR-left caud
SNR_CAUDL_sd = []                           # create frame for sd of left caud
voxmean_CAUDL = []

SNR_data_CAUDR = []                         # create frame for SNR-right caud
SNR_CAUDR_sd = []                           # create frame for sd of right caud 
voxmean_CAUDR = []

county = 0
list_of_files = []

for img in range(0,len(data_sheet)):

    ## Updated: Script for SNR & CNR calculations
    # take 3 x 3 x 3 square around voxel coordinate in ROI (CC or caudate) 
    # 27 voxel measurements in total for each image 

    county += 1
    
    for root, dirs, files in os.walk(data_dir):
        for file in files:
            if file.endswith(file_name[img]):
                file_path = os.path.join(root, file)
    
    list_of_files.append(file_path)
    if (len(list_of_files) > 3) and (list_of_files[county-1]==list_of_files[county-2]):
        print('n/n/n/Warning, same file used twice')
    
    print(file_name[img])
    print(str(county) + ': ' + file_path)                
    
    # Load data 
    hdr = nib.load(file_path) # load image data for CC and CAUD SNRs
    data = hdr.get_data() # extract and rename it to data

    mean_data = data # not fMRI so dont need to mean acorss volumes

    ###################################################
    # DATA FOR CC
    x = CCx[img] # insert voxel coordinate x 
    y = CCy[img] # insert voxel coordinate y 
    z = CCz[img] # insert voxel coordinate z 

    voxel_points = [x,y,z]
    voxel_data = []
    
    print(voxel_points)
    
    DAT = spherical(x,y,z,voxel_data,mean_data)
    
    stand = DAT[-1]        # standard deviation
    mean_vox = DAT[-2]     # mean voxel intensity
    SNR = mean_vox/stand   # SNR
    
    SNR_data_CC.append(SNR)
    SNR_CC_sd.append(stand)
    voxmean_CC.append(mean_vox)
    
    ###################################################
    # DATA FOR LEFT CAUDATE
    x = CAUDLx[img] # insert voxel coordinate x 
    y = CAUDLy[img] # insert voxel coordinate y 
    z = CAUDLz[img] # insert voxel coordinate z 

    voxel_points = [x,y,z]
    voxel_data = []
    
    DAT = spherical(x,y,z,voxel_data,mean_data)
    
    stand = DAT[-1]
    mean_vox = DAT[-2]
    SNR = mean_vox/stand
    
    SNR_data_CAUDL.append(SNR)
    SNR_CAUDL_sd.append(stand)
    voxmean_CAUDL.append(mean_vox)
    
    ###################################################
    # DATA FOR RIGHT CAUDATE
    x = CAUDRx[img] # insert voxel coordinate x 
    y = CAUDRy[img] # insert voxel coordinate y 
    z = CAUDRz[img] # insert voxel coordinate z 

    voxel_points = [x,y,z]
    voxel_data = []
    
    DAT = spherical(x,y,z,voxel_data,mean_data)
    
    stand = DAT[-1]
    mean_vox = DAT[-2]
    SNR = mean_vox/stand
    
    SNR_data_CAUDR.append(SNR)
    SNR_CAUDR_sd.append(stand)
    voxmean_CAUDR.append(mean_vox)

In [ ]:
# convert data_sheet to pandas
data_labels_pd = pd.DataFrame(data_sheet.Label)

In [ ]:
# info for CC
SNR_data_CC
SNR_data_CC_pd = pd.DataFrame(SNR_data_CC) # convert to pandas
SNR_data_CC_pd.columns = ['CC']            # set column name 

SNR_sd_CC_pd = pd.DataFrame(SNR_CC_sd)
SNR_sd_CC_pd.columns = ['CC_sd']

mean_vox_CC_pd = pd.DataFrame(voxmean_CC)
mean_vox_CC_pd.columns = ['CC_meanvox']

In [ ]:
# Info for left CN
SNR_data_CAUDL
SNR_data_CAUDL_pd = pd.DataFrame(SNR_data_CAUDL) # convert to pandas 
SNR_data_CAUDL_pd.columns = ['CAUDL']            # set column name

SNR_sd_CAUDL_pd = pd.DataFrame(SNR_CAUDL_sd)
SNR_sd_CAUDL_pd.columns = ['CAUDL_sd']

mean_vox_CAUDL_pd = pd.DataFrame(voxmean_CAUDL)
mean_vox_CAUDL_pd.columns = ['CAUDL_meanvox']

In [ ]:
# Info for right CN
SNR_data_CAUDR
SNR_data_CAUDR_pd = pd.DataFrame(SNR_data_CAUDR) # convert to pandas 
SNR_data_CAUDR_pd.columns = ['CAUDR']            # set column name

SNR_sd_CAUDR_pd = pd.DataFrame(SNR_CAUDR_sd)
SNR_sd_CAUDR_pd.columns = ['CAUDR_sd']

mean_vox_CAUDR_pd = pd.DataFrame(voxmean_CAUDR)
mean_vox_CAUDR_pd.columns = ['CAUDR_meanvox']

In [ ]:
# Concatenate dataframe to fill with SNRs
#vox_details = pd.concat([data_sheet.Database_sub_ses,data_sheet.Label,SNR_data_CC_pd,SNR_data_CAUDL_pd,SNR_data_CAUDR_pd],axis=1)
vox_details = pd.concat([data_sheet.Database,data_sheet.Label,SNR_data_CC_pd,SNR_data_CAUDL_pd,SNR_data_CAUDR_pd],axis=1)
vox_details['Label'] = vox_details['Label'].astype("category") # set to type category 

In [ ]:
vox_details['CNR'] = vox_details['CC'] - vox_details['CAUDL']
vox_details['CNR'] = vox_details['CNR'].abs()

In [ ]:
summary_vox = vox_details[['CC','CAUDL','CAUDR','CNR']]
#summary_vox.to_excel('summary_vox.xlsx')

In [ ]:
pd.options.display.max_rows = 4000
# mean left caudate SNR + mean_vox + sd
pd.concat([vox_details.groupby('Label')['CAUDL'].mean(),vox_details.groupby('Label')['CAUDL'].std()],axis=1)
# mean right caudate SNR + mean_vox + sd
pd.concat([vox_details.groupby('Label')['CAUDR'].mean(),vox_details.groupby('Label')['CAUDR'].std()],axis=1)
# mean cc SNR + mean_vox + sd
pd.concat([vox_details.groupby('Label')['CC'].mean(),vox_details.groupby('Label')['CC'].std()],axis=1)
# mean CNR 
pd.concat([vox_details.groupby('Label')['CNR'].mean(),vox_details.groupby('Label')['CNR'].std()],axis=1)

In [ ]:
# allinfo = pd.concat([data_sheet.Database_sub_ses,data_labels_pd,SNR_data_CC_pd,SNR_data_CAUDL_pd,
#                      SNR_data_CAUDR_pd],axis=1)

allinfo = pd.concat([data_sheet.Database,data_labels_pd,SNR_data_CC_pd,SNR_sd_CC_pd,mean_vox_CC_pd,SNR_data_CAUDL_pd,SNR_sd_CAUDL_pd,mean_vox_CAUDL_pd,
                     SNR_data_CAUDR_pd,SNR_sd_CAUDR_pd,mean_vox_CAUDR_pd],axis=1)

In [ ]:
# save dataframe to csv to input to R scripts for analysis
allinfo.to_csv(os.path.join(data_dir, 'SNRinfo_all_20.csv'))